In [ ]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import scipy.stats as ss
import math
import random
import numba
from collections import namedtuple


import time


plt.style.use('classic') #'seaborn-whitegrid'
sns.set()
%precision 4
%matplotlib inline
# To reload modules
import importlib


from FileHandling import save_state, inspect_state, save_run_data, save_comparison


from MH_Sampling import acceptance_decision, MH_sampler, L_MH_sampler, init_MH_pars
#from SG_MH_Sampling import SG_MH_sampler, init_SG_MH_pars
from AM_Sampling import AM_sampler, init_AM_pars
from CMA_Sampling import CMA_sampler, init_CMA_pars
from GaA_Sampling import C_GaA_sampler, Q_GaA_sampler, init_GaA_pars


from TestSuite import generate_state_space, generate_iid_samples, \
    generate_initial_states, generate_Gaussian, get_distribution, get_samples



SAMPLERS = {'MH':dict(Name='MH', Function=MH_sampler, 
                      InitParameters=init_MH_pars, 
                      Color='red'),
            'L_MH':dict(Name='L_MH', Function=L_MH_sampler, 
                        InitParameters=init_MH_pars, 
                        Color='brown'),
            'AM':dict(Name='AM', 
                      Function=AM_sampler, 
                      InitParameters=init_AM_pars, 
                      Color='blue'),
            'CMA':dict(Name='CMA', 
                       Function=CMA_sampler, 
                       InitParameters=init_CMA_pars, 
                       Color='green'),
            'GaA':dict(Name='GaA', 
                       Function=Q_GaA_sampler, 
                       InitParameters=init_GaA_pars,
                       Color='yellow')}

#'SG_MH':dict(Name='SG_MH', Function=SG_MH_sampler, InitParameters=init_SG_MH_pars, Color='black')



def get_sampler(name):
    global SAMPLERS
    return SAMPLERS[name]



def create_data_store(dim, N):
    return dict(States=np.zeros(shape=(N, dim)),
                Densities=np.zeros(N), 
                Accepted_p=np.zeros(N, dtype=bool),
                Means=np.zeros(shape=(N, dim)),
                # Covariance matrix or its Cholesky decomposition or square root
                Covariances=np.zeros((N, dim, dim)), 
                # The parameter r in Gaussian Adaption or sigma in CMA-ES is stored in
                Scales=np.zeros(N), 
                # The parameter theta in Gaussian Adaption is stored in
                Thresholds=np.zeros(N), 
                # Done is false when created and becomes true when the run is finished
                Done=False,
                # ChainLength is 0 when created and is set to the effective chain length
                # when the run is finished.
                ChainLength=0)



RunData = namedtuple('RunData',
                     ['StateSpace', 'N', 'Target', 'Sampler', 'RunIdx', 'DataStore'])



def specify_run(dim, N, name_target, name_sampler, run_idx):
    sp = generate_state_space(dim=dim)
    generate_iid_samples(sp=sp, N=N)
    target = get_distribution(sp=sp, name=name_target)
    sampler = get_sampler(name=name_sampler)
    data_store = create_data_store(dim=dim, N=N)
    return RunData(StateSpace=sp, 
                   N=N, 
                   Target=target, 
                   Sampler=sampler, 
                   RunIdx=run_idx, 
                   DataStore=data_store)



def get_states(run_data):
    return run_data.DataStore['States']

def get_contour_function_and_levels(run_data):
    target = run_data.Target
    return target['Contour Function'], target['Contour Levels']



def execute_run(run_data):
    sp, sampler, N, run_idx = \
        run_data.StateSpace, run_data.Sampler, run_data.N, run_data.RunIdx
    initial_states = sp['Initial States']
    return sampler['Function'](pars=sampler['InitParameters'](sp=sp),
                               target=run_data.Target, 
                               initial_state=initial_states[run_idx], 
                               run_data=run_data)


tic = time.time()

# specify dimensions 'dim', choose sampler 'name_sampler', choose target distribution 'namme_target'
SPEC = specify_run(dim=2, N=1800, name_target='Pi_4', name_sampler='MH', run_idx=0)
DATA = execute_run(SPEC)

toc = time.time()

print('Run time: ',  str((toc - tic)) + 'seconds')

from Visualization import GRID, plot_samples, compare_to_iid_samples, plot_contour_lines, compare_sample_spread



compare_to_iid_samples(run_data=DATA, dim1=0, dim2=1, nb_samples=2, burnin_pct=0)

In [ ]:
DATA